In [11]:
import torch
import faulthandler
from IPython.display import clear_output

from vctr.data.lstm_preprocessor import preprocess_data
from vctr.data.data_loader import get_data_with_features_and_labels
from vctr.models.lstm.utils import get_new_model_name
from vctr.models.lstm.actions import new_model, save_model, train_model, load_model
from vctr.trading.coins import trainable_coins
from vctr.models.echo import EchoStateNetwork

faulthandler.enable()
torch.autograd.set_detect_anomaly(True)

input_size = 482
num_classes = 3
dropout_rate = 0.3
num_layers = 2
hidden_size = 128

model = EchoStateNetwork(input_dim=input_size, reservoir_size=2000, output_dim=num_classes)

data = get_data_with_features_and_labels(
    symbol='ACH',
    timeframe='15m',
    separate=False,
    label_args=(0.05, 0.005),
)

X_train, y_train, X_test, y_test = preprocess_data(
    data.drop(['label'], axis=1), data['label'], lookback=24, test_pct=0.25, torch=True
)

model.train(X_train, y_train)


/opt/homebrew/Caskroom/miniforge/base/envs/v2/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [12]:
from sklearn.metrics import classification_report

y_pred = model.predict(X_test)

# Generate classification report
report = classification_report(y_test, y_pred)
print('Classification Report:')
print(report)


Classification Report:
              precision    recall  f1-score   support

           0       0.99      0.96      0.97     23217
           1       0.32      0.65      0.42       413
           2       0.39      0.64      0.49       349

    accuracy                           0.95     23979
   macro avg       0.57      0.75      0.63     23979
weighted avg       0.97      0.95      0.96     23979



In [15]:
data = get_data_with_features_and_labels(
    symbol='BTC',
    timeframe='15m',
    separate=False,
    label_args=(0.05, 0.005),
)

X_train, y_train, X_test, y_test = preprocess_data(
    data.drop(['label'], axis=1), data['label'], lookback=24, test_pct=0.25, torch=True
)


In [16]:
y_pred = model.predict(X_train)

# Generate classification report
report = classification_report(y_train, y_pred)
print('Classification Report:')
print(report)

Classification Report:
              precision    recall  f1-score   support

           0       0.96      0.94      0.95     41966
           1       0.17      0.28      0.21      1112
           2       0.13      0.14      0.14      1105

    accuracy                           0.90     44183
   macro avg       0.42      0.45      0.43     44183
weighted avg       0.92      0.90      0.91     44183

